Anomaly-Detection-in-time-series/Anomaly-Detection-in-Time-Series.ipynb

https://github.com/Charlie5DH/Anomaly-Detection-in-time-series/blob/master/Anomaly-Detection-in-Time-Series.ipynb

In [ ]:
#전처리 과정
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(train[['close']])

train['close'] = scaler.transform(train[['close']])
test['close'] = scaler.transform(test[['close']])

Build LSTM autoencoder

In [ ]:
timesteps = X_train.shape[1]
num_features = X_train.shape[2]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

model = Sequential([
    LSTM(128, input_shape=(timesteps, num_features)),
    Dropout(0.2),
    RepeatVector(timesteps), # replicates features from outputs (30 times)
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    # Time distributed layer to get an output with right shape
    TimeDistributed(Dense(num_features))
])
model.compile(loss='mae', optimizer='adam')
model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[es],
    shuffle=False
)

In [ ]:

model = Sequential()
model.add(LSTM(..., input_shape=(...)))
model.add(RepeatVector(...))
model.add(LSTM(..., return_sequences=True))
model.add(TimeDistributed(Dense(...)))

https://github.com/takanyanta/Sparse-LSTM-Autoencoder-Implementation/blob/main/Sparse_LSTM_autoencoder.ipynb

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Flatten, Reshape, Embedding,ActivityRegularization, Lambda, Input,Multiply,Activation
from tensorflow.keras.models import Model
import changefinder
import seaborn as sns
from scipy.spatial import distance
import random
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPool2D, Conv1D
from tensorflow.python.keras.layers import Layer, InputSpec
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
pd.set_option('display.max_columns', 100)

In [ ]:
class preprocess:
    def create_dataset(self, X, y, time_steps=1):
        self.X = X
        self.y = y
        self.Xs, self.ys = [], []
        self.time_steps = time_steps
        for self.i in range(len(self.X) - self.time_steps):
            self.v = self.X.iloc[self.i:(self.i + self.time_steps)].values
            self.Xs.append(self.v)        
            self.ys.append(self.y.iloc[self.i + self.time_steps])
        return np.array(self.Xs), np.array(self.ys)
    
    def Creating_Models(self, timesteps, num_features, hidden = 10):
        self.hidden = hidden
        self.timesteps = timesteps
        self.num_features = num_features
        self.model = Sequential([
            #Encoder
            LSTM(self.hidden, input_shape=(self.timesteps, self.num_features)),
            #Dropout(0.2),
            #Decoder
            RepeatVector(self.timesteps),
            LSTM(self.hidden, return_sequences=True),
            #Dropout(0.2),
            TimeDistributed(Dense(self.num_features))                 
        ])
        return self.model
    
    def plotting_history(self, history):
        self.history = history
        plt.figure(figsize=(4, 4))
        plt.plot(self.history.history['loss'], label='Training Loss')
        plt.plot(self.history.history['val_loss'], label='Validation Loss')
        plt.xlabel("Epochs")
        plt.ylabel("loss")
        plt.legend()
        plt.show()

In [ ]:
def Usual_LSTM(X):
    hidden = 5
    timesteps=X.shape[1]
    num_features=X.shape[2]
    model = Sequential([
        LSTM(hidden, input_shape=(timesteps, num_features)),
        RepeatVector(timesteps),
        LSTM(hidden, return_sequences=True),
        TimeDistributed(Dense(num_features))                 
    ])
    model.summary()
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
class MyLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    def build(self, input_shape):
        self.kernel = self.add_weight(
        "kernel", shape=[1, self.units],
        initializer='uniform', trainable=True,
        regularizer = tf.keras.regularizers.l1(0.001)
        )
    def call(self, input):
        output = input*self.kernel
        return tf.nn.relu(output) 

def Sparse_LSTM(X):
    hidden = 5
    timesteps=X.shape[1]
    num_features=X.shape[2]
    model = Sequential([
        Flatten(input_shape=(timesteps, num_features)),
        MyLayer(timesteps*num_features),
        Reshape(target_shape=(timesteps, num_features)),
        LSTM(hidden, input_shape=(timesteps, num_features)),
        RepeatVector(timesteps),
        LSTM(hidden, return_sequences=True),
        TimeDistributed(Dense(num_features))    
    ])
    model.compile( loss="mse", optimizer='adam')
    model.summary()
    return model

In [ ]:
train = create_df(1).iloc[:10000]
test = create_df(1).iloc[10000:]
sc = StandardScaler()
train_std = pd.DataFrame(sc.fit_transform(train), columns=train.columns)
test_std = pd.DataFrame(sc.transform(test), columns=test.columns)

X_train, _ = preprocess().create_dataset(train_std, train_std, time_steps=15)
X_test , _ = preprocess().create_dataset(test_std, test_std, time_steps=15)

In [ ]:
model = Usual_LSTM(X_train)